# Hypotézy

1. Loajálnym zákazníkom menej prekáža horší inflight service ako neloajálnym zákazníkom.
2. Ľuďom nad 50 rokov menej prekáža zlé pokrytie wifi počas letu.
3. Ak je pasažier počas dlhého letu nespokojný s komfortom, tak je nespokojný aj s celým letom.
4. Počas dlhých letov je pasažierom ponúkané lepšie jedlo.
5. Čím dlhšie meškanie odletu, tak tým dlhšie meškanie príletu (hypotéza so spojitými dátami)


In [1]:
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import random
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('dataset/final_df.csv', index_col='Unnamed: 0')

train_df, test_df = train_test_split(df, test_size=0.2, shuffle=True)

In [3]:
def tests(x, y):
    for _ in range(20):
        x = random.sample(x, 100)
        y = random.sample(y, 100)

        print(chisquare(x, f_exp=y))

In [4]:
def log_reg(X, y):
    clf = LogisticRegression(random_state=0).fit(X, y)
    return clf

def dec_tree(X, y):
    clf = tree.DecisionTreeClassifier().fit(X, y)
    return clf

def rand_forest(X, y):
    clf = RandomForestClassifier(max_depth=3, random_state=0).fit(X, y)
    return clf

def acc(model, X, y_true):
    y_pred = model.predict(X)
    print(accuracy_score(y_true, y_pred))

def one_hot(data, col):
    enc = pd.get_dummies(data[col])
    data = data.drop(col,axis = 1)
    data = data.join(enc)
    return data

## 1. Loajálnym zákazníkom prekáža horší inflight service rovnako alebo viac ako neloajálnym zákazníkom

Na základe dát chceme zistiť, ako ovplyvňuje kvalita služieb počas letu celkovú spokojnosť loajálnych a neloajálnych zákazníkov. Náš predpoklad je taký, že horšia kvalita služieb počas letu vadí loajálnym zákazníkov rovnako alebo viac ako neloajálnym. Na základe predpokladu sme si stanovili nulovú a alternativnu hypotézu.

### Nul. Loajálnym zákazníkom prekáža horší inflight service rovnako alebo viac ako neloajálnym zákazníkom.

### Alt. Loajálnym zákazníkom prekáža horší inflight service menej ako neloajálnym zákazníkom.

#### Postup pri overovaní hypotézy

Na začiatok sme vyselektovali spokojnosť loajálnych a neloajálnych zákazníkov, ktorí neboli spokojní so službami počas letu. Ako nespokojných sme označili všetkých pasažierov, ktorí ohodnotili služby počas letu 3 a menej na škále od 1 do 5. Pričom 1 značí najmenšiu spokojnosť.

Následne sme si vykreslili stĺpcové grafy, na ktorých sme zobrazili pomer celkovej spokojnosti s letom zákazníkov, ktorí neboli spokojní so službami počas letu. 

Prvý graf znázorňuje loajálnych zákazníkov. Vidíme v ňom, že až 69.9% loajálnych zákazníkov, ktorí boli nespokojní so službami počas letu, boli celkovo nespokojní aj s letom.

Druhý graf znázorňuje neloajálnych zákazníkov. Vidíme v ňom, že až dokonca 89.1% neloajálnych zákazníkov, ktorí boli nespokojní so službami počas letu, boli celkovo nespokojní aj s letom.

Už z grafov môžeme predpokladať, že neloajálni zákazníci sú menej tolerantní a viacej im vadia zlé služby počas letu.

Ďalej sme na overenie nášho predpokladu vykonali chisquare test. Tento test sme vybrali, pretože dáta, s ktorými pracujeme sú kategorické. Výsledná p-hodnota tohto testu bola menšia ako 0.001, čiže pravdepodobnosť chyby prvého rádu je menšia ako jedno promile. Tým pádom na základe chisquare testu odmietame nulovú hypotézu a akceptujeme alternativnu hypotézu.

Na koniec sme na vybraných dátach natrénovali štatistický model (Random Forest), ktorý mal na základe typu zákazníka a spokojnosti so službami počas letu určiť celkovú spokojnosť zákazníka s letom. Na testovacej množine sme dosiahli úspešnosť 64.98%. 

In [5]:
loyal_customer = df.loc[(df['Customer Type'] == 'Loyal Customer') & (df['Inflight service'] <= 3), 'satisfaction']
disloyal_customer = df.loc[(df['Customer Type'] == 'disloyal Customer') & (df['Inflight service'] <= 3), 'satisfaction']

In [ ]:
loyal_customer.value_counts().sort_index().plot(kind='bar')

loyal = loyal_customer.value_counts().sort_index()
loyal / sum(loyal.values.tolist()) * 100

In [ ]:
disloyal_customer.value_counts().sort_index().plot(kind='bar')

disloyal = disloyal_customer.value_counts().sort_index()
disloyal / sum(disloyal.values.tolist()) * 100

In [ ]:
chisquare(loyal.values.tolist(), f_exp=disloyal.values.tolist())

In [ ]:
train_X = train_df[['Customer Type', 'Inflight service']]
train_y = train_df['satisfaction']

test_X = test_df[['Customer Type', 'Inflight service']]
test_y = test_df['satisfaction']

train_X = one_hot(train_X, 'Customer Type')
test_X = one_hot(test_X, 'Customer Type')

model = rand_forest(train_X, train_y)
acc(model, test_X, test_y)

## 2. Zákazníkom nad 50 rokov prekáža zlé pokrytie wifi rovnako alebo viac ako zákazníkom pod 50 rokov

Na základe dát chceme zistiť, ako ovplyvňuje pokrytie wifi celkovú spokojnosť zákazníkov nad a pod 50 rokov. Náš predpoklad je taký, že horšie pokrytie wifi prekáža zákazníkom nad 50 rokov rovnako alebo viac ako zákazníkom pod 50 rokov. Na základe predpokladu sme si stanovili nulovú a alternativnu hypotézu.

### Nul. zákazníkom nad 50 rokov prekáža zlé pokrytie wifi rovnako alebo viac ako zákazníkom pod 50 rokov.

### Alt. zákazníkom nad 50 rokov prekáža zlé pokrytie wifi počas letu menej ako zákazníkom pod 50 rokov.

Na začiatok sme vyselektovali spokojnosť zákazníkov nad a pod 50 rokov, ktorí neboli spokojní s pokrytím wifi. Ako nespokojných sme označili všetkých zákazníkov, ktorí ohodnotili pokrytie wifi 3 a menej na škále od 1 do 5. Pričom 1 značí najmenšiu spokojnosť.

Následne sme si vykreslili stĺpcové grafy, na ktorých sme zobrazili pomer celkovej spokojnosti s letom zákazníkov, ktorí neboli spokojní s pokrytím wifi.

Prvý graf znázorňuje zákazníkov nad 50 rokov. Vidíme v ňom, že až 65.3% zákazníkov nad 50 rokov, ktorí boli nespokojní s pokrytím wifi, boli celkovo nespokojní aj s letom.

Druhý graf znázorňuje zákazníkov pod 50 rokov. Vidíme v ňom, že až dokonca 71.7% zákazníkov pod 50 rokov, ktorí boli nespokojní s pokrytím wifi, boli celkovo nespokojní aj s letom.

Už z grafov môžeme predpokladať, že zákazníci nad 50 rokov sú menej zaťažený na pokrytie wifi, tým pádom vplýva menej na celkovú spokojnosť s letom ako u zákazníkov pod 50 rokov.

Ďalej sme na overenie nášho predpokladu vykonali chisquare test. Tento test sme vybrali, pretože dáta, s ktorými pracujeme sú kategorické. Výsledná p-hodnota tohto testu bola menšia ako 0.001, čiže pravdepodobnosť chyby prvého rádu je menšia ako jedno promile. Tým pádom na základe chisquare testu odmietame nulovú hypotézu a akceptujeme alternativnu hypotézu.

Na koniec sme na vybraných dátach natrénovali štatistický model (Rozhodovací Strom), ktorý mal na základe veku zákazníka a spokojnosti s pokrytím wifi určiť celkovú spokojnosť zákazníka s letom. Na testovacej množine sme dosiahli úspešnosť 75.45%. 

In [ ]:
older_than_fifty = df.loc[(df['Age'] >= 50) & (df['Inflight wifi service'] <= 3), 'satisfaction']
younger_than_fifty = df.loc[(df['Age'] < 50) & (df['Inflight wifi service'] <= 3), 'satisfaction']

In [ ]:
older_than_fifty.value_counts().plot(kind='bar')

older = older_than_fifty.value_counts().sort_index()
older / sum(older.values.tolist()) * 100

In [ ]:
younger_than_fifty.value_counts().plot(kind='bar')

younger = younger_than_fifty.value_counts().sort_index()
younger / sum(younger.values.tolist()) * 100

In [ ]:
chisquare(older.values.tolist(), f_exp=younger.values.tolist())

In [ ]:
train_X = train_df[['Age', 'Inflight wifi service']]
train_y = train_df['satisfaction']

test_X = test_df[['Age', 'Inflight wifi service']]
test_y = test_df['satisfaction']

model = dec_tree(train_X, train_y)
acc(model, test_X, test_y)

## 3 Pasažieri, ktorí sú počas dlhého letu nespokojní s komfortom, tak sú spokojní s celým letom rovnako alebo viac ako tí čo sú spokojní s komfortom.

Na základe dát chceme zistiť, ako ovplyvňuje kvalita komfortu celkovú spokojnosť zákazníkov počas dlhých letov. Náš predpoklad je taký, že horšia kvalita komfortu pri dlhých letoch má vplyv na celkovú spokojnosť s letom. Na základe predpokladu sme si stanovili nulovú a alternativnu hypotézu.

### Nul. Pasažieri, ktorí sú počas dlhého letu nespokojní s komfortom, tak sú spokojní s celým letom rovnako alebo viac ako tí čo sú spokojní s komfortom.

### Alt. Pasažieri, ktorí sú počas dlhého letu spokojní s komfortom, tak sú spokojní s celým letom viac ako tí čo nie sú spokojní s komfortom.

Na začiatok sme vyselektovali spokojnosť zákazníkov, ktorí neboli spokojní s komfortom a tých, ktorí boli spokojní počas dlhých letov. Ako dlhé lety sme klasifikovali tie, ktorých dĺžka bola nad hodnotou mediánu (843). Ako nespokojných sme označili všetkých pasažierov, ktorí ohodnotili komfort na 3 a menej. Ako spokojných sme označili všetkých pasažierov, ktorí ohodnotili komfort na 4 a 5. Hodnoty sú na škále od 1 do 5. Pričom 1 značí najmenšiu spokojnosť.

Následne sme si vykreslili stĺpcové grafy, na ktorých sme zobrazili pomer celkovej spokojnosti s letom u zákazníkov, ktorí neboli a boli spokojní s komfortom počas dlhých letov.

Prvý graf znázorňuje zákazníkov, ktorí boli nespokojní s komfortom počas dlhých letov. Vidíme v ňom, že až dokonca 75.5% zákazníkov, ktorí boli nespokojní s komfortom počas dlhých letov, boli celkovo nespokojní aj s letom.

Druhý graf znázorňuje zákazníkov, ktorí boli spokojní s komfortom počas dlhých letov. Vidíme v ňom, že iba 28.4% zákazníkov, ktorí boli spokojní s komfortom, boli celkovo nespokojní aj s letom.

Už z grafov môžeme predpokladať, že komfort počas dlhých letov má významný vplyv na celkovú spokojnosť s letom.

Ďalej sme na overenie nášho predpokladu vykonali chisquare test. Tento test sme vybrali, pretože dáta, s ktorými pracujeme sú kategorické. Výsledná p-hodnota tohto testu bola menšia ako 0.001, čiže pravdepodobnosť chyby prvého rádu je menšia ako jedno promile. Tým pádom na základe chisquare testu odmietame nulovú hypotézu a akceptujeme alternativnu hypotézu.

Na koniec sme na vybraných dátach natrénovali štatistický model (Random Forest), ktorý mal na základe spokojnosti s komfortom počas dlhých letov určiť celkovú spokojnosť zákazníka s letom. Na testovacej množine sme dosiahli úspešnosť 70.89%. 

In [ ]:
long_flights_discomfort = df.loc[(df['Flight Distance'] > 843) & (df['Seat comfort'] <= 3), 'satisfaction']
long_flights_comfort = df.loc[(df['Flight Distance'] > 843) & (df['Seat comfort'] > 3), 'satisfaction']

In [ ]:
long_flights_discomfort.value_counts().sort_index().plot(kind='bar')

discomfort = long_flights_discomfort.value_counts().sort_index()
discomfort / sum(discomfort.values.tolist()) * 100

In [ ]:
long_flights_comfort.value_counts().sort_index().plot(kind='bar')

comfort = long_flights_comfort.value_counts().sort_index()
comfort / sum(comfort.values.tolist()) * 100

In [ ]:
chisquare(comfort.values.tolist(), f_exp=discomfort.values.tolist())

In [ ]:
train_X = train_df[['Flight Distance', 'Seat comfort']]
train_y = train_df['satisfaction']

test_X = test_df[['Flight Distance', 'Seat comfort']]
test_y = test_df['satisfaction']

model = rand_forest(train_X, train_y)
acc(model, test_X, test_y)

model = dec_tree(train_X, train_y)
acc(model, test_X, test_y)

model = log_reg(train_X, train_y)
acc(model, test_X, test_y)

## 4 Počas dlhých letov je pasažierom ponúkané lepšie jedlo ako je ponúkané počas krátkych letov.
Za dlhé lety sú považované tie, ktorých vzdialenosť je väčšia ako medián.

### Nul. Počas dlhých letov je pasažierom ponúkané horšie/rovnako kvalitné jedlo ako je ponúkané počas krátkych letov.

### Alt. Počas dlhých letov je pasažierom ponúkané kvalitnejšie jedlo ako je ponúkané počas krátkych letov.

Na základe chisquare testu akceptujeme alternativnu hypotezu.

In [ ]:
long_flights = df.loc[df['Flight Distance'] > 843, 'Food and drink']
short_flights = df.loc[df['Flight Distance'] <= 843, 'Food and drink']

In [ ]:
long_flights.value_counts().sort_index().plot(kind='bar')

long_f = long_flights.value_counts().sort_index()
long_f / sum(long_f.values.tolist()) * 100

In [ ]:
short_flights.value_counts().sort_index().plot(kind='bar')

short = short_flights.value_counts().sort_index()
short / sum(short.values.tolist()) * 100

In [ ]:
chisquare(long_f.values.tolist(), f_exp=short.values.tolist())

In [ ]:
train_X = train_df[['Flight Distance', 'Food and drink']]
train_y = train_df['satisfaction']

test_X = test_df[['Flight Distance', 'Food and drink']]
test_y = test_df['satisfaction']

model = rand_forest(train_X, train_y)
acc(model, test_X, test_y)

model = dec_tree(train_X, train_y)
acc(model, test_X, test_y)

model = log_reg(train_X, train_y)
acc(model, test_X, test_y)

## 5 Rozdiel dĺžky meškania odletu pasažierov, ktorí sú spokojní s letom a pasažierov, ktorí sú nespokojní s letom je signifikantný.

### Nul Rozdiel dĺžky meškania odletu pasažierov, ktorí sú spokojní s letom a tých, ktorých sú nespokojní s letom nie je signigikantný.

### Alt Rozdiel dĺžky meškania odletu pasažierov, ktorí sú spokojní s letom a tých, ktorých sú nespokojní s letom je signigikantný.

Na základe Mann Whitney U-testu akceptujeme nulovú hypotézu.

In [ ]:
delay_dissatisfied = df.loc[(df['satisfaction'] == 'neutral or dissatisfied'), 'Departure Delay in Minutes']
sns.distplot(delay_dissatisfied, bins=30)

stats.shapiro(delay_dissatisfied)

In [ ]:
delay_satisfied = df.loc[(df['satisfaction'] == 'satisfied'), 'Departure Delay in Minutes']
sns.distplot(delay_satisfied, bins=30)

stats.shapiro(delay_satisfied)

In [ ]:
for _ in range(20):
    x = random.sample(delay_dissatisfied.tolist(), 100)
    y = random.sample(delay_satisfied.tolist(), 100)

    print(stats.mannwhitneyu(x, y))


In [ ]:
train_X = train_df[['Departure Delay in Minutes']]
train_y = train_df['satisfaction']

test_X = test_df[['Departure Delay in Minutes']]


test_y = test_df['satisfaction']

model = rand_forest(train_X, train_y)
acc(model, test_X, test_y)

model = dec_tree(train_X, train_y)
acc(model, test_X, test_y)

model = log_reg(train_X, train_y)
acc(model, test_X, test_y)